In [25]:
from tensorflow.keras.layers import MaxPooling2D, Conv2D
import numpy as np
import tensorflow as tf

# Step 2 (30 points)
def net48_siso(): 
    height, width = 48, 48
    inputs = tf.keras.Input((height, width, 3));
    x = Conv2D(8,3,1,'same',activation='relu')(inputs)
    # enter your code here
     
    x = MaxPooling2D(2, 2)(x)#對卷積層應用2*2的pooling


    x = Conv2D(16, 3, 1, 'same', activation='relu')(x)
    x = MaxPooling2D(2, 2)(x)


    x = Conv2D(32, 3, 1, 'same', activation='relu')(x)
    x = MaxPooling2D(2, 2)(x)


    x = Conv2D(64, 3, 1, 'same', activation='relu')(x)
    x = MaxPooling2D(2, 2)(x)

    
    x = Conv2D(2, 3, 1, 'valid', activation='softmax', name='cls_output')(x)




    outputs = x
    return tf.keras.Model(inputs = inputs, outputs = outputs)


In [26]:
# Step 3 (15 points)
def net48_simo(): 
    height, width = 48, 48
    inputs = tf.keras.Input((height, width, 3));
    x = Conv2D(8,3,1,'same',activation='relu')(inputs)
    # enter your code here

    x = MaxPooling2D(2, 2)(x)

    x = Conv2D(16, 3, 1, 'same', activation='relu')(x)
    x = MaxPooling2D(2, 2)(x)

    x = Conv2D(32, 3, 1, 'same', activation='relu')(x)
    x = MaxPooling2D(2, 2)(x)

    x = Conv2D(64, 3, 1, 'same', activation='relu')(x)
    x = MaxPooling2D(2, 2)(x)

    cls_output = Conv2D(2, 3, 1, 'valid', activation='softmax', name='cls_output')(x) #cls = np.reshape(outputs[0], (2,))
    reg_output = Conv2D(4, 3, 1, 'valid', name='reg_output')(x) #reg = np.reshape(outputs[1], (4,))
    return tf.keras.Model(inputs=inputs, outputs=[cls_output, reg_output])

In [27]:
# Do not change below.
def check_model(cnn_model, is_simo=False):
  print('Checking the CNN model...')
  is_passed = True
  is_cls_output = False
  is_reg_output = False
  ishape = cnn_model.input_shape[1:]
  if (48,48,3) != ishape:
     print('Error! The input shape', ishape, 'is not equal to (48,48,3)')
     is_passed = False
  for i,layer in enumerate(cnn_model.layers):
     if layer.name == 'cls_output':
       oshape = layer.output_shape[1:]
       is_cls_output = True
       if (1,1,2) != oshape:
          print('Error! The output shape', oshape, 'is not equal to (1,1,2)')
          is_passed = False
     if layer.name == 'reg_output':
       oshape = layer.output_shape[1:]
       is_reg_output = True
       if (1,1,4) != oshape:
          print('Error! The output shape', oshape, 'is not equal to (1,1,4)')
          is_passed = False
     layer_type = layer.__class__.__name__
     if layer_type == 'Dense':
       print('Error! Cannot use Dense layers.')
       is_passed = False
     if layer_type == 'Conv2D':
       kernel_size = layer.kernel_size
       strides = layer.strides
       if np.max(kernel_size) > 7:
         print('({:d} {:s}) Error! kernel_size should be less than or equal to 7'.format(i, layer_type))
         is_passed = False
       if np.max(strides) > 2:
         print('({:d} {:s}) Error! stride should be less than or equal to 2'.format(i, layer_type))
         is_passed = False
     if layer_type == 'MaxPooling2D':
       pool_size = layer.pool_size
       strides = layer.strides
       if np.max(pool_size) > 2:
         print('({:d} {:s}) Error! pool_size should be less than or equal to 2'.format(i, layer_type))
         is_passed = False
       if np.max(strides) > 2:
         print('({:d} {:s}) Error! stride should be less than or equal to 2'.format(i, layer_type))
         is_passed = False
  if not is_cls_output:
     print('Error! There should be a Conv. layer with name cls_output')
     is_passed = False
  if is_simo:
     if not is_reg_output:
       print('Error! There should be a Conv. layer with name reg_output')
       is_passed = False
  if is_passed:
     print('Great! The CNN model satisfies all the requirements.')


In [28]:
# Do not change below.
model48_siso = net48_siso()
model48_siso.summary()
check_model(model48_siso)
print()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 48, 48, 8)         224       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 24, 24, 8)         0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 24, 24, 16)        1168      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 12, 12, 16)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 12, 12, 32)        4640      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 6, 6, 32)          0   

In [29]:
# Do not change below.
model48_simo = net48_simo()
model48_simo.summary()
check_model(model48_simo, is_simo=True)
print()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 48, 48, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_23 (Conv2D)              (None, 48, 48, 8)    224         input_10[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_19 (MaxPooling2D) (None, 24, 24, 8)    0           conv2d_23[0][0]                  
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (None, 24, 24, 16)   1168        max_pooling2d_19[0][0]           
____________________________________________________________________________________________

In [30]:
# Do not change below.
img = np.random.randn(1, 48, 48, 3)
outputs = model48_simo.predict(img)
print(outputs[0].shape) # should display (1,1,1,2)
print(outputs[1].shape) # should display (1,1,1,4)
cls = np.reshape(outputs[0], (2,))
reg = np.reshape(outputs[1], (4,))
print(cls)

(1, 1, 1, 2)
(1, 1, 1, 4)
[0.6007426 0.3992575]


In [31]:
# Question 2 (5 points)
# Why cls[0] + cls[1] is always equal to one regardless of  the values of img?

#我們使用了 softmax， softmax會將 cls_output 轉換為0~1的機率，cls[0] 和 cls[1] 分別表示兩類別的機率，因此能夠確保輸出總合為1。